<a href="https://colab.research.google.com/github/SotaYoshida/Lecture_DataScience/blob/main/notebooks/Python_chapter_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Web操作・スクレイピング

Webから情報を抽出・整形・解析したり、ブラウザ上での特定の操作を自動化する、といったこともPythonでは比較的容易に実行することができる。

Web上にある情報にアクセスしたりする方法は色々あるが、大まかには２つに分類され

1. プログラムでWebページにアクセスして中身(HTML)を読み、そこから情報を抽出する方法
2. ブラウザをプログラムに操作させて特定の作業を実行する方法

この章では、とくに1.のWebから情報を抽出すること(スクレイピング)に絞ってそのエッセンスを紹介する。  
HTMLとは、Webページを作るための言語のことで、みなさんが普段からアクセスする多くのWebページは、HTMLで記述されています。

Webから情報を抽出したりする際、共通して言える注意点を述べておく:
* **対象とするページの利用規約を必ず確認する**  
  規約でスクレイピングを禁止しているページがある  
  (例: Amazon, Twitter, Instagram, facebook, 金融系などなど)  
  禁止している場合でも、APIが提供されている場合がある  
  (※APIはApplication Programming Interfaceの略)  
  今の場合、大雑把にはデータ提供用の窓口とでも思ってください.

* **サーバーに負荷をかけない**  
  規約で特にスクレイピングを禁止していない場合でも、過度なアクセスをしてはいけません。  
  (どこかの大学の教務ポータルみたいに落ちてしまったら大変です)   
  過度なアクセスは、悪意のあるDos攻撃とみなされてアクセスを制限されたり、  
  最悪の場合、偽計業務妨害罪などの罪に問われる可能性があります。






## 東京都の新型コロナウイルス関連情報の取得

スクレイピングを可能とするライブラリは多数存在する。  
代表的なものは```requests```や```urllib```。以下では```requests```を使います。  

JavaScriptの実行などがないページならこれでだいたい十分かと思います.  
また```BeutifulSoup4```というライブラリを使うと、HTMLを構文解析して、  
タグなどを取り除くといった操作が比較的容易に行えます。



1つ目の例として、[東京都 新型コロナウイルス感染症対策サイト](https://stopcovid19.metro.tokyo.lg.jp/)の公開データを取得して  
それをグラフにして可視化してみましょう。まずは必要なライブラリをインポートしておきます。

In [ ]:
import requests
import json #json形式のデータを扱うために必要
import datetime #日付の処理に使う
import matplotlib.pyplot as plt
!pip install japanize-matplotlib 
import japanize_matplotlib 

[東京都 新型コロナウイルス感染症対策サイト](https://stopcovid19.metro.tokyo.lg.jp/)の[レポジトリ](https://github.com/tokyo-metropolitan-gov/covid19)で公開されているデータ(json形式)を読み出す。

指定したurlにリクエストを送り、レスポンスをjson形式で取得する

>ちなみに、json形式のデータは、元々JavaScriptのオブジェクトとして扱うために作られたものであるが、
PythonやJava, PHPなどでも扱うことができる。Pythonで言うと辞書型が近い。

In [ ]:
url = "https://raw.githubusercontent.com/tokyo-metropolitan-gov/covid19/development/data/data.json"
response = requests.get(url)
data = response.json()

上の`data`(辞書型)を直接`print`すると表示が冗長になるため、代わりに、キーとして何があるかを見てみよう。

In [ ]:
data.keys()

それぞれ
* contacts: 新型コロナコールセンター相談件数
* querents: 帰国者・接触者電話相談センター相談件数
* patients_summary: 陽性患者数
* inspections_summary: 検査実施日別状況
* lastUpdate: 最終更新日時
* main_summary: (これはよくわからない, 使用されていなかったっぽい)

に対応している。

陽性患者数のデータの構造を調べつつ結果を可視化してみよう。  
まずは型を調べてみる。

In [ ]:
type(data['patients_summary'])

辞書型とわかったのでキーの一覧を見てみよう

In [ ]:
data['patients_summary'].keys()

同様にして`data['patients_summary']["data"]`が"日付"と"小計"をキーに持つ辞書のリストだと分かる。

In [ ]:
type(data['patients_summary']['data'])

In [ ]:
data['patients_summary']["data"][0]

日付と陽性患者数をリストに直して、描画してみよう

In [ ]:
#　"2022-01-01"のような文字列をdatetimeに変換する関数
def str_to_dt(tstr):
    tdatetime = datetime.datetime.strptime(tstr, '%Y-%m-%d')
    tdate = datetime.date(tdatetime.year, tdatetime.month, tdatetime.day)
    return tdate
    
tdat = data['patients_summary']["data"]
N = len(tdat)
tdat = data['patients_summary']["data"]
dates = [ str_to_dt(tdat[i]["日付"].split("T")[0]) for i in range(N)]
num_patients = [ tdat[i]['小計'] for i in range(N)]
print("dates=>",  dates)
print("陽性患者数=>", num_patients)

In [ ]:
fig = plt.figure(figsize=(15,4))
plt.title("東京都")
plt.xlabel("日付")
plt.ylabel("陽性者数[人]")
plt.plot(dates,num_patients, color="green")
plt.show()
plt.close()

このようにして、Web上のデータを取得して、可視化することができた。

ちなみに、東京都のCOVID19の[レポジトリ](https://github.com/Tokyo-Metro-Gov/covid19)は、
5類以降後の2023年6月にPublic archiveとなり、更新を停止している。

## e-Statのデータを取得する

[政府統計の総合窓口(e-Stat)](https://www.e-stat.go.jp/)は日本の各府省が公表する統計データを一つにまとめた政府統計のポータルサイトである。

e-Statでは[API機能](https://www.e-stat.go.jp/api/)(ユーザー登録が必要)が提供されていて、  
指定されたURLに対してリクエストを送信することで各APIを利用することが出来る。




### 準備1 e-Statの登録






以降の内容でも、e-StatのAPI機能を使用するため予めe-Statへの登録が必要になる。

国内の公開データを使って最終課題に取り組みたいという方は、必ず、ここに記載の手順にならって登録しておこう。

1. まず[こちら](https://www.e-stat.go.jp/mypage/user/preregister)のページからメールアドレス(Gmailを推奨します)を入力し、届いたメールから本登録を行う  
  このとき、gmailアカウントによる認証をオンにしておくと、以後のログインが楽になる。

2. [ログインページ](https://www.e-stat.go.jp/mypage/login)からログインし、右上の[マイページ]に遷移
  <img src="https://drive.google.com/uc?export=view&id=1NHMQ8GnJ6n6hSaUAIFgOGyRc5Z55Enxj" width = 70%>

3. [API機能(アプリケーションID発行)]に遷移する

4. 以下の例にならって、名称とURLを入力し、発行ボタンを押す。  
  ここでは、名称はGoogleColab、URLは`https://test.localhost`としている。  
  <img src="https://drive.google.com/uc?export=view&id=100w2nmie7ZD6PliQVbjAHIt7o5gaBciN" width = 80%>

5. [appId]に生成された文字列を、どこかに控えておく(**appIdを他人と共有してはいけません**)


### 準備2 Pandas e-Statの導入


さて、e-Statへの登録ができたでしょうか。  

早速e-Statにあるデータをどんどん抽出したい  
...のだが、[マニュアル](https://www.e-stat.go.jp/api/sites/default/files/uploads/2019/07/API-specVer3.0.pdf)を理解してAPIに適切なリクエストを送るというのは、それ自体がややハードルが高いように思う。

そこで、このノートブックではPython用のライブラリ[Pandas e-Stat](https://github.com/kotamya/pandas-estat)を使用することにする。  
ただし、このライブラリについてもpublic archive化されていて`pip`などでインストールできないため、
Google Colabで使用しているサーバー上にライブライのソースファイルをダウンロードして展開し、インポートする荒業で対応する。  


In [ ]:
#ホームディレクトリにレポジトリをクローン
%cd 
!git clone https://github.com/kotamya/pandas-estat.git

# ディレクトリ名を変更し、レポジトリ内に移動しインポート
!mv pandas-estat local_library
%cd ./local_library/
import pandas_estat as pandas_estat

**AppIDの定義**

In [ ]:
appID = "dcda7e96569055203230f2aaf838de5200aace1e" 
pandas_estat.set_appid(appID)


```read_statslist()```関数の引数に"政府統計コード"を指定することで、  
該当する統計データの一覧を```pandas.DataFrame```形式で取得することができる。  
[政府統計コードの一覧はこちら](https://www.e-stat.go.jp/help/stat-search-3-5)

以下では、「サービス産業動向調査」の結果を例に抽出してみる。

In [ ]:
# サービス産業動向調査
statslist = read_statslist("00200544")  
statslist

次に、これらのデータがどうやって分類されているのかを調べてみましょう。  
列(column)を表示してみると

In [ ]:
print(statslist.columns)

例えば、```CYCLE```(周期)というものがあるので、  
CYCLEが"月次"(月ごと)になっているデータだけをピックアップして、  

In [ ]:
statslist = statslist[statslist.CYCLE == "月次"] 


その```TABLE_INF```(統計表ID)と```TITLE```(タイトル)だけを表示してみましょう

In [ ]:
statslist[["TABLE_INF", "TITLE"]]

```TABLE_INF```の数値が[統計表ID]と呼ばれるものです。

ではこの中から"事業活動の産業（中分類）別売上高（月次）【2013年1月～】"を読み出してみましょう。  
それには```read_statsdata```という関数が用意されているので、  
引数に```TABLE_INF```(統計表ID)を入力すると...

In [ ]:
from pandas_estat import read_statsdata 
df = read_statsdata("0003191203")  
df

これで、データを抽出することができました。

In [ ]:
df.columns

もう属性を少し絞ってみましょう。

dataframeの"事業活動の産業"列にどんなものがあるかを抽出し、  
```set```関数を使って重複を除くと...

In [ ]:
set(df["事業活動の産業"])

"42鉄道業"を見てみましょう。  

In [ ]:
ndf = df[df["事業活動の産業"] == "42鉄道業"] #dfの中の、"事業活動の産業欄が"42鉄道業"のものを抽出し、ndfと名前をつける

ndf = ndf[ndf["時間軸（月次）"].str.endswith("月")]
ndf["時間軸（月次）"] = pd.to_datetime(ndf["時間軸（月次）"], format="%Y年%m月")
ndf = ndf.sort_values("時間軸（月次）")

#必要なところだけ抽出すると...
ndf[["時間軸（月次）", "value", "unit"]]

図を描いてみよう

In [ ]:
import matplotlib.pyplot as plt
!pip install japanize-matplotlib 
import japanize_matplotlib 
import matplotlib.dates as mdates

In [ ]:
x = ndf["時間軸（月次）"].values
y = ndf["value"].values.astype(float) * 1.e-2  # XX億円に換算

fig = plt.figure(figsize=(30, 4))
ax = fig.add_subplot(111)  
ax.set_facecolor("#e0e0e0")
ax.set_ylabel("鉄道業の収益 [億円]")
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=range(1, 13, 4)))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y/%m"))
ax.grid(True,axis="both",color="w", linestyle="dotted", linewidth=0.8)
ax.plot(x,y,marker="o",color="green")
plt.show()
plt.close()

新型コロナウイルスの感染拡大を受けて2020年の1月ごろから  
緊急事態宣言の解除(5月25日)まで、大幅に収益が減っていることが分かる。



---



### 備考

あらかじめ興味のあるデータ,分類・属性等がわかっている場合は、  
上記のコードの対応するIDなどを変更すればよいし  
コードを拡張して、データ,分類・属性等の一覧表などを適宜表示しながら、  
特定のキーワードを含むものに絞ってデータを取得したりもできる。

ちなみに、GoogleColabで提供されているコードスニペット(テンプレートのようなもの)  
を用いることで、プルダウンでデータ,分類・属性等を選んでプロットする、  
といったようにGoogle Colab.上で自分だけのe-Stat分析ツールを作ることもできる  
(最終課題で取り組むのも良い)


In [ ]:
!git clone https://github.com/kotamya/pandas-estat.git
!mv pandas-estat local_library
%cd local_library


In [ ]:
!pwd
#import local_library.pandas_estat

In [ ]:
import requests
import pandas as pd

# グローバル変数でAppIDを設定する
appID = "" # 引用符内を準備1で発行したappidに置き換える
appID = "dcda7e96569055203230f2aaf838de5200aace1e" 



In [ ]:
from typing import Any
from typing import Dict
from typing import Optional

def read_statslist(code):
    # code: str,
    # limit: int = None,
    # start_position: Optional[int] = None,
    # updated_date: Optional[str] = None,
    # lang: str = "J",
    # appid: Optional[str] = None,
    # version: str = "3.0",
    # **kwargs,
    # pd.DataFrame:

    reader = StatsListReader(
        code,
        limit=limit,
        start_position=start_position,
        updated_date=updated_date,
        lang=lang,
        appid=appid,
        version=version,
    )
    dataframe = reader.read(**kwargs)
    return dataframe

class StatsListReader(BaseReader):
    """統計表情報を取得します。

    Args:
        code (str): 政府統計コードです。
            数値 5 桁: 作成機関で検索
            数値 8 桁: 政府統計コードで検索
            次のページから検索できます。
            https://www.e-stat.go.jp/api/api-info/api-data
            e-Stat API の ``statsCode`` に対応します。
        limit (int, optional): データの取得行数を指定して下さい。
            省略時は 10 万件です。
            データ件数が指定した limit 値より少ない場合、全件を取得します。
            データ件数が指定した limit 値より多い場合（継続データが存在する）は、
            受信したデータの ``<NEXT_KEY>`` タグに継続データの開始行が設定されます。
            e-Stat API の ``limit`` に対応します。
        start_position (int, optional): データの取得開始位置（1 から始まる行番号）を指定して下さい。
            省略時は先頭から取得します。
            統計データを複数回に分けて取得する場合等、継続データを取得する開始位置を指定するために指定します。
            前回受信したデータの ``<NEXT_KEY>`` タグの値を指定します。
            e-Stat API の ``startPosition`` に対応します。
        updated_date (str, optional): 更新日付を指定します。
            指定された期間で更新された統計表の情報を提供します。
            以下のいずれかの形式で指定して下さい。

            - ``YYYY``: 単年検索
            - ``YYYYMM``: 単月検索
            - ``YYYYMMDD``: 単日検索
            - ``YYYYMMDD-YYYYMMDD``: 範囲検索

            e-Stat API の ``updatedDate`` に相当します。
        lang ({"J", "E"}, default "J"): 取得するデータの言語です。
            ``"J"`` （日本語）または ``"E"`` （英語）で指定してください。
            e-Stat API の ``lang`` に対応します。
        appid (str, optional): アプリケーション ID です。
            指定しない場合、:func:`pandas_estat.set_appid` で指定した値か、
            環境変数 ``ESTAT_APPID`` を用います。
            e-Stat API の ``appId`` に対応します。
        version (str, default "3.0"): API 仕様バージョンです。
            https://www.e-stat.go.jp/api/api-info/api-spec
    """

    QUERY = "getSimpleStatsList"
    TABLE_TAG = "STAT_INF"

    def __init__(
        self,
        code: str,
        limit: Optional[int] = None,
        start_position: Optional[int] = None,
        updated_date: Optional[str] = None,
        lang: str = "J",
        appid: Optional[str] = None,
        version: str = "3.0",
    ) -> None:
        self.code = code
        self.limit = limit
        self.start_position = start_position
        self.updated_date = updated_date
        self.lang = lang
        self.appid = get_appid(appid)
        self.version = version

        if self.appid is None:
            raise ValueError("アプリケーション ID が指定されていません。")
        if not (isinstance(code, str) and re.fullmatch(r"(\d{5}|\d{8})", code)):
            raise ValueError(
                "政府統計コードは 5 桁か 8 桁の数字を str 型で指定してください。\n"
                "e-Stat 提供データ: https://www.e-stat.go.jp/api/api-info/api-data"
            )

        if lang == "E":
            raise NotImplementedError
        if lang not in ("J", "E"):
            raise ValueError('言語 lang は "J" か "E" で指定してください。')

    @property
    def params(self) -> dict:
        """e-Stat API のパラメータ群を `dict` 形式で返します。

        Note:
            - 参照: e-Stat API v3.0 仕様 2.1 API の利用方法 - 統計表情報取得
            - 参照: e-Stat API v3.0 仕様 3.2 API パラメータ - 統計表情報取得

        Returns:
            dict
        """
        params: Dict[str, Any] = {"appId": self.appid, "statsCode": self.code}

        if self.limit is not None:
            params["limit"] = self.limit
        if self.start_position is not None:
            params["startPosition"] = self.start_position
        if self.updated_date is not None:
            params["updatedDate"] = self.updated_date
        if self.lang is not None:
            params["lang"] = self.lang

        return params